# Nicolas Dorval

<h1 style="text-align:center">Travail pratique numérique en thermodynamique statistique</h1>
<h2 style="text-align:center">PARTIE 2 : Modèle de Drude</h2>

Veuillez indiquer le nom des membres de votre équipe dans la cellule suivante.

- Pierre-Olivier Desrosiers<br>
- Nicolas Dorval<br>
- Gérémy Michaud<br>
- Félix-Antoine Dupuis

# Introduction # 
Cette deuxième partie du travail implémente une simulation 2D du modèle de Drude pour décrire le mouvement des électrons libres et indépendants dans les métaux. Des sphères dures sont encore utilisées pour représenter les particules, mais maintenant de deux types différents afin de différencier les coeurs ioniques immobiles des électrons sur leurs trajectoires balistiques. Les collisions entre les deux doivent donc être inélastiques. Les questions sur cette simulation, d'abord de cinétique puis de dynamique des électrons en présence d'une force externe appliquée au cristal, vérifieront quelques résultats analytiques du modèle de Drude obtenus en classe et/ou dans le manuel de cours Ashcroft/Mermin.

- **La remise du présent _Jupyter Notebook_ ET celui de la 1<sup>re</sup> partie, ainsi que tout autre fichier de code produit, se fait sur Gradescope en n'oubliant pas d'y indiquer tous les membres de votre équipe.**

In [1]:
from numpy import *
import random
from vpython import *
from IPython.display import display, Math
import math
import matplotlib.pyplot as plt
import time

<IPython.core.display.Javascript object>

Faites une copie du script `TDS-2Dsimulation_HXX.py` et modifiez-le pour obtenir une simulation cinétique d'un gaz d'électrons libres dans la matière cristalline selon le modèle de Drude. Spécifiquement selon les pp.4-6 du manuel Ashcroft/Mermin,
1. ajoutez un deuxième type de sphères fixes réparties périodiquement dans la boîte d'animation: celles-ci représenteront les coeurs ioniques,
2. éliminez les collisions entre les sphères mobiles qui représentent maintenant les électrons de conduction indépendants et libres,
3. en faisant appel à la température du gaz, ajoutez des collisions inélastiques entre les électrons libres et les coeurs ioniques fixes. La quantité de mouvement $\vec{p}$ n'est alors PAS conservée et il faut appliquer les hypothèses spécifiques du modèle de Drude à la sortie de chaque collision, notamment: 
- la direction de $\vec{p}$ doit être aléatoire,
- la norme $||\vec{p}||$ est déterminée par la distribution de Maxwell-Boltzmann.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

### Votre simulation ###

VII. _(25 points)_ &mdash; Écrivez un appel de votre nouveau script pour l'exécuter avec la cellule suivante:

In [ ]:
#Code de GM (test4) modifié légèrement pour la question 10
# Created on Fri Feb 5 15:40:27 2021
# GlowScript 3.0 VPython

# Hard-sphere gas.

# Bruce Sherwood
# Claudine Allen

from vpython import *
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.stats import maxwell

def Simulation(champ, iterations):
    # Déclaration de variables influençant le temps d'exécution de la simulation
    Natoms = 100  # change this to have more or fewer atoms
    Ncoeur_coter = 8
    dt = 1E-5  # pas d'incrémentation temporel

    # Déclaration de variables physiques "Typical values"
    mass = 4E-3/6E23  # helium mass
    Ratom = 0.01  # wildly exaggerated size of an atom
    Rcoeur = 0.03
    k = 1.4E-23  # Boltzmann constant # TODO: changer pour une constante de Boltzmann en eV/K :)
    T = 300  # around room temperature
    q = 1.602e-19

    # CANEVAS DE FOND
    L = 1  # container is a cube L on a side
    gray = color.gray(0.7)  # color of edges of container and spheres below
    animation = canvas(width=750, height=500)  # , align='left')
    animation.range = L

    # ARÊTES DE BOÎTE 2D
    d = L/2+Ratom
    r = 0.005
    cadre = curve(color=gray, radius=r)
    cadre.append([vector(-d, -d, 0), vector(d, -d, 0), vector(d, d, 0), vector(-d, d, 0), vector(-d, -d, 0)])

    # POSITION ET QUANTITÉ DE MOUVEMENT INITIALE DES SPHÈRES
    Atoms = []  # Objet qui contiendra les sphères pour l'animation
    Coeur = []  # Objet qui contiendra les
    p = []  # quantité de mouvement des sphères
    apos = []  # position des sphères
    cpos = []  # position des coeurs
    pavg = sqrt(2*mass*1.5*k*T)  # average kinetic energy p**2/(2mass) = (3/2)kT

    # Création de coordonée pour les sphère fixes
    for i in range(Ncoeur_coter):  # Nombre de ligne
        for n in range(Ncoeur_coter):  # Nombre de ligne
            x = i*L/(Ncoeur_coter-1)-L/2  # position d'un coeur en tenant compte de l'origine au centre de la boîte
            y = n*L/(Ncoeur_coter-1)-L/2
            z = 0
            Coeur.append(simple_sphere(pos=vector(x, y, z), radius=Rcoeur, color=color.red))
            cpos.append(vec(x, y, z))

    for i in range(Natoms):
        x = L*random()-L/2  # position aléatoire qui tient compte que l'origine est au centre de la boîte
        y = L*random()-L/2
        z = 0
        if i == 0:
            Atoms.append(simple_sphere(pos=vector(x, y, z), radius=0.03, color=color.magenta))
        else:
            Atoms.append(simple_sphere(pos=vector(x, y, z), radius=Ratom, color=gray))
        apos.append(vec(x, y, z))

        phi = 2*pi*random()  # direction aléatoire pour la quantité de mouvement
        px = pavg*cos(phi)  # qte de mvt initiale selon l'équipartition
        py = pavg*sin(phi)
        pz = 0
        p.append(vector(px, py, pz))  # liste de la quantité de mouvement initiale de toutes les sphères

    # FONCTION POUR IDENTIFIER LES COLLISIONS
    def checkCollisions():
        hitlist = []  # initialisation
        r2 = Ratom + Rcoeur  # distance critique où les 2 sphères entre en contact à la limite de leur rayon
        r2 *= r2
        for i in range(Ncoeur_coter**2):
            ci = cpos[i]
            for j in range(Natoms):
                aj = apos[j]
                dr = ci - aj
                if mag2(dr) < r2:
                    hitlist.append([i, j])
        return hitlist

    # BOUCLE PRINCIPALE POUR L'ÉVOLUTION TEMPORELLE DE PAS dt
    it = 0
    collision = 0
    liste_p = []
    liste_p_avg = []
    liste_pos_avg_y = []
    liste_pos_avg_x = []

    for i in range(iterations):
        rate(100)  # limite la vitesse de calcul de la simulation pour que l'animation soit visible à l'oeil humain!

        pavg = np.mean(np.array([nb_p.mag for nb_p in p]))  # remodifie la valeur du p average pour chaque itéreation
        posavgy = np.mean(np.array([pos.y for pos in apos]))
        posavgx = np.mean(np.array([pos.x for pos in apos]))


        # DÉPLACE TOUTES LES SPHÈRES D'UN PAS SPATIAL deltax
        vitesse = []  # vitesse instantanée de chaque sphère
        deltax = []  # pas de position de chaque sphère correspondant à l'incrément de temps dt
        for i in range(Natoms):
            vitesse.append(p[i]/mass)
            deltax.append(vitesse[i] * dt)
            Atoms[i].pos = apos[i] = apos[i] + deltax[i]

        # CONSERVE LA QUANTITÉ DE MOUVEMENT AUX COLLISIONS AVEC LES MURS DE LA BOÎTE
        for i in range(Natoms):
            loc = apos[i]
            if abs(loc.x) > L/2:
                if loc.x < 0:
                    p[i].x = abs(p[i].x)
                else:
                    p[i].x = -abs(p[i].x)
            if abs(loc.y) > L/2:
                if loc.y < 0:
                    p[i].y = abs(p[i].y)
                else:
                    p[i].y = -abs(p[i].y)
                p[i].y += q * champ * dt

        # LET'S FIND THESE COLLISIONS!!!
        hitlist = checkCollisions()

        liste_p_avg.append(pavg)
        liste_pos_avg_y.append(posavgy)
        liste_pos_avg_x.append(posavgx)
        collision += len(hitlist)

        temperature = pavg**2/(3*k*mass)
        # CONSERVE LA QUANTITÉ DE MOUVEMENT AUX COLLISIONS ENTRE SPHÈRES
        for ij in hitlist:
            i = ij[1]
            j = ij[0]
            posi = apos[i]
            posj = cpos[j]
            vi = p[i]/mass
            rrel = hat(posi-posj)
            vrel = vi

            dx = dot(posi-posj, vrel.hat)
            dy = cross(posi-posj, vrel.hat).mag
            alpha = asin(dy/(Ratom+Rcoeur))
            d = (Ratom+Rcoeur)*cos(alpha)-dx
            deltat = d/vrel.mag

            # CHANGE L'INTERPÉNÉTRATION DES SPHÈRES PAR LA CINÉTIQUE DE COLLISION
            angle = pi*(random()-1/2)
            norm_p = mass*maxwell.rvs(scale=sqrt(k*temperature/mass))
            p[i] = norm_p * hat(rotate(rrel, angle=angle))
            apos[i] = posi+(posi-posj)*deltat

            liste_p.append(p[i].mag)

        it += 1
    return [liste_p_avg, liste_pos_avg_y, liste_pos_avg_x]

In [ ]:
nbre_iter = 700
liste_p_avg, liste_pos_avg_y, liste_pos_avg_x = Simulation(0, nbre_iter)

### Question statistique ###

VIII. _(5 points)_ &mdash; Vérifiez numériquement et graphiquement que l'amortissement de la quantité de mouvement moyenne des électrons suit l'équation exponentielle dérivée analytiquement en classe, soit $\langle p(t)\rangle =\langle p(t_0)\rangle \,e^{-t/\tau}$, et comparez-y le comportement d'un seul électron.

In [ ]:
iterations = np.linspace(0, nbre_iter, nbre_iter)
p_init = liste_p_avg[0]
modele = lambda t, tau: p_init*np.exp(-t/tau)
popt, pcov = curve_fit(modele, iterations, liste_p_avg)
plt.figure(figsize=(8, 6))
plt.plot(iterations, liste_p_avg, label="Simulation")
plt.plot(iterations, modele(iterations, popt), label="Modèle")
plt.legend()
plt.xlabel("Itérations")
plt.ylabel("p(t) [kg m/s]")
plt.show()

### Dynamique sous l'effet d'une force externe ###

IX. _(10 points)_ &mdash; Pour passer de la cinétique à la dynamique des électrons libres, modifiez votre code de simulation en ajoutant une fonction qui applique un champ électrique uniforme. Celui-ci devra être de module ajustable et perpendiculaire à deux des côtés de la boîte. À chaque pas de temps $\mathrm{d}t$ sans collision, les électrons devront donc accélérer d'un incrément $\mathrm{d}p_x$ dicté par la force de Coulomb à leur position.

Copiez le code de votre fonction dans la cellule qui suit en n'oubliant pas d'inclure votre fichier Python (`.py`) modifié avec la simulation complète lors de la remise.

### Question statistique ###

X. _(5 points)_ &mdash; Pour quelques différents modules de champ électrique, présentez graphiquement l'évolution de la position moyenne des électrons en fonction du temps pour ses deux composantes parallèle et perpendiculaire au champ.

In [ ]:
liste_p_avg_1, liste_pos_avg_y_1, liste_pos_avg_x_1 = Simulation(0, nbre_iter)
liste_p_avg_2, liste_pos_avg_y_2, liste_pos_avg_x_2 = Simulation(1, nbre_iter)
liste_p_avg_3, liste_pos_avg_y_3, liste_pos_avg_x_3 = Simulation(2, nbre_iter)

iterations = np.linspace(0, nbre_iter, nbre_iter)

fig, axs = plt.subplots(2)
plt.subplots_adjust(left=0.15, bottom=0.1, right=0.95, top=0.9, wspace=0.4, hspace=0.5)

axs[0].plot(iterations, liste_pos_avg_y_1)
axs[0].plot(iterations, liste_pos_avg_y_2)
axs[0].plot(iterations, liste_pos_avg_y_3)

axs[1].plot(iterations, liste_pos_avg_x_1)
axs[1].plot(iterations, liste_pos_avg_x_2)
axs[1].plot(iterations, liste_pos_avg_x_3)

axs[0].legend(["Champ électrique nul", "Champ électrique de module 1", "Champ électrique de module 2"], loc="upper left")
axs[0].set_xlabel("Itérations")
axs[0].set_ylabel("Position moyenne")
axs[0].title.set_text("Position moyenne parallèle (y) au champ électrique")

axs[1].legend(["Champ électrique nul", "Champ électrique de module 1", "Champ électrique de module 2"], loc="upper left")
axs[1].set_xlabel("Itérations")
axs[1].set_ylabel("Position moyenne")
axs[1].title.set_text("Position moyenne parallèle (x) au champ électrique")
plt.show()

# Médiagraphie #
- P. Drude, _Zur Elektronentheorie der Metalle; I Teil_, Annalen der Physik **306**(3), pp.566–613 (1900) https://doi.org/10.1002/andp.19003060312
- P. Drude, _Zur Elektronentheorie der Metalle; II Teil. Galvanomagnetische und Thermomagnetische Effecte_, Annalen der Physik **308**(11), pp.369–402 (1900) https://doi.org/10.1002/andp.19003081102
- P. Drude, _Zur Elektronentheorie der Metalle; Berichtigung_, Annalen der Physik **312**(3), pp.687–692 (1902) https://doi.org/10.1002/andp.19023120312
- H. A. Lorentz, _The Motion of Electrons in Metallic Bodies I_, Proc. of Koninklijke Akademie van Wetenschappen **7**, pp.438-453 (1905) https://dwc.knaw.nl/DL/publications/PU00013989.pdf
- H. A. Lorentz, _The Motion of Electrons in Metallic Bodies II_, Proc. of Koninklijke Akademie van Wetenschappen **7**, pp.585-593 (1905) https://dwc.knaw.nl/DL/publications/PU00014010.pdf
- H. A. Lorentz, _The Motion of Electrons in Metallic Bodies III_, Proc. of Koninklijke Akademie van Wetenschappen **7**, pp.684-691 (1905) https://dwc.knaw.nl/DL/publications/PU00014024.pdf
- La simulation utilise la librairie <a href="https://vpython.org">VPython</a> conçue pour faciliter la visualisation de physique en 3D, avec les instructions d’installation <a href="https://vpython.org/presentation2018/install.html">ici</a> et la documentation <a href="https://www.glowscript.org/docs/VPythonDocs/index.html">ici</a>. Le code adapte en 2D et commente en détail l’exemple <a href="https://www.glowscript.org/#/user/GlowScriptDemos/folder/Examples/program/HardSphereGas-VPython">HardSphereGas-VPython</a> du site interactif <a href="https://www.glowscript.org">GlowScript</a> pour programmer des animations avec VPython directement en ligne.